### 은닉 마르코프 모형 (HMM) - 기초

*이론 강의의 HMM 모형을 만들어 본다. 첨부된 'fig_hmm.png'를 참고해 본다.*

In [ ]:
# 다음 라이브러리를 별도로 설치해야 한다.
# 설치에 시간이 상당히 소요된다 (~5분).
!pip install pomegranate==0.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pomegranate: filename=pomegranate-0.12.0-cp310-cp310-linux_x86_64.whl size=14204226 sha256=9afc3adefa8c831a7e22c0d2125d1f5677dc556b0720d953c10a875ace1cafc5
  Stored in directory: /root/.cache/pip/wheels/2b/70/cb/9cdb862af960baf0fc3c2c461456ee721ef08a14b76c6935bb
Successfully built pomegranate
  Attempting uninstall: pomegranate
    Found existing installation: pomegranate 1.0.0
    Uninstalling pomegranate-1.0.0:
      Successfully uninstalled pomegranate-1.0.0


In [ ]:
# 라이브러리를 불러온다.
import numpy as np
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

#### 1. 모델 파라미터 설정.

1.1. 상태의 초기 확률 (Initial Probabilities):

|        | Prob |
|: --- :|: --- :|
| $Sunny$  | 0.50 |
|$Rainny$  | 0.50 |

1.2. 전이 확률 (Transition Probabilities):

|   | $Sunny$ | $Rainny$ |
|:---:|:---:|:---:|
|$Sunny$| 0.80 | 0.40 |
|$Rainny$| 0.20 | 0.60 |

1.3. 출력 확률 (Emission Probabilities):

|    |  $Sunny$  | $Rainny$ |
|: ---: |: --- :|: ---: |
| $Game$ |   0.10  | 0.80 |
| $Fitness$ | 0.90 | 0.20 |


#### 2. 모델 생성.

In [ ]:
# 모델 생성.
my_hmm = HiddenMarkovModel(name="my_model")

In [ ]:
# 상태와 출력확률 P(O | S) 설정.
Sunny_emissions = DiscreteDistribution({"Game": 0.1, "Fitness": 0.9})
Sunny_state = State(Sunny_emissions, name="Sunny_state")

Rainny_emissions = DiscreteDistribution({"Game": 0.8, "Fitness": 0.2})
Rainny_state = State(Rainny_emissions, name="Rainny_state")

# 모델에 상태 추가.
my_hmm.add_states(Sunny_state)
my_hmm.add_states(Rainny_state)

In [ ]:
# 초기 확률 설정.
my_hmm.add_transition(my_hmm.start, Sunny_state, 0.5)
my_hmm.add_transition(my_hmm.start, Rainny_state, 0.5)

In [ ]:
# 전이 확률 P(S'|S) 설정.
my_hmm.add_transition(Sunny_state, Sunny_state, 0.8)  # 80% Sunny -> Sunny
my_hmm.add_transition(Sunny_state, Rainny_state, 0.2)  # 20% Sunny -> Rainny

my_hmm.add_transition(Rainny_state, Sunny_state, 0.4)  # 40% Rainny -> Sunny
my_hmm.add_transition(Rainny_state, Rainny_state, 0.6)  # 60% Rainny -> Rainny

In [ ]:
# 모델 최종 완성.
my_hmm.bake()

In [ ]:
# 입력된 상태의 이름 확인.
for s in my_hmm.states:
    print(s.name)

Rainny_state
Sunny_state
my_model-start
my_model-end


#### 3. 모델을 사용한 예측 (Inference).

In [ ]:
# 다음과 같은 시퀀스가 관찰 되었다 전제한다.
observations = ["Game", "Fitness", "Game"]

In [ ]:
# Viterbi 알고리즘으로 가장 우도 (likelihood)가 높은 시퀀스를 계산한다.
viterbi_log_likelihood, viterbi_path = my_hmm.viterbi(observations)

In [ ]:
# 제일 유력한 날씨의 시퀀스.
weather_pred = []
for s in viterbi_path[1:]:
    weather_pred.append(s[1].name)
print("제일 유력한 날씨 : {}".format(weather_pred))
print("우도 함수의 값   : {:0.4f}".format(np.exp(viterbi_log_likelihood)))

제일 유력한 날씨 : ['Rainny_state', 'Sunny_state', 'Rainny_state']
우도 함수의 값   : 0.0230
